In [3]:
import wget

In [3]:
url_conferencia_j = "https://amlo.presidente.gob.mx/30-09-24-version-estenografica-de-la-ultima-conferencia-de-prensa-del-presidente-andres-manuel-lopez-obrador/"
wget.download(url=url_conferencia_j, out="conferencia_j.txt")

'conferencia_j.txt'

In [4]:
url_pagina_i = "https://amlo.presidente.gob.mx/secciones/version-estenografica/page/153/"
wget.download(url=url_pagina_i, out="pagina_i.txt")

'pagina_i.txt'

In [5]:
import re
import glob
import io

In [5]:
url_pagina_i = "https://amlo.presidente.gob.mx/secciones/version-estenografica/page/"
num_paginas = 153

for i in range(num_paginas):
    wget.download(url=url_pagina_i + str(i), out="./paginas/" + str(i+1) + ".txt")

In [10]:
urls_conf = []

for f_pagina in glob.glob("./paginas/*"):
    #print(f_pagina)
    urls_conf += [ path for protocol, domain, path in 
                  re.findall('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])', 
                             open(f_pagina, "r", encoding="utf-8").read())
                             if "version-estenografica-de-la" in path]

In [11]:
urls_conf

['/secciones/version-estenografica/feed/',
 '/16-02-20-version-estenografica-inauguracion-de-instalaciones-de-la-guardia-nacional-desde-penjamo-guanajuato/',
 '/16-02-20-version-estenografica-inauguracion-de-instalaciones-de-la-guardia-nacional-desde-penjamo-guanajuato/',
 '/15-02-20-version-estenografica-inauguracion-de-instalaciones-de-la-guardia-nacional-en-lagos-de-moreno-jalisco/',
 '/15-02-20-version-estenografica-inauguracion-de-instalaciones-de-la-guardia-nacional-en-lagos-de-moreno-jalisco/',
 '/15-02-20-version-estenografica-inauguracion-de-instalaciones-de-la-guardia-nacional-en-tepatitlan-de-morelos-jalisco/',
 '/15-02-20-version-estenografica-inauguracion-de-instalaciones-de-la-guardia-nacional-en-tepatitlan-de-morelos-jalisco/',
 '/14-02-20-version-estenografica-nauguracion-de-instalaciones-de-la-guardia-nacional-desde-sahuayo-de-morelos-michoacan/',
 '/14-02-20-version-estenografica-nauguracion-de-instalaciones-de-la-guardia-nacional-desde-sahuayo-de-morelos-michoacan/',

In [14]:
urls_conf_set = list(set(urls_conf))

In [16]:
for path in urls_conf_set:
    f = wget.download(url="https://amlo.presidente.gob.mx" + path, 
                      out="./estenograficas/" + path.replace("/","-"))[0]

In [20]:
from bs4 import BeautifulSoup

for f_pagina in glob.glob("./estenograficas/*"):
    soup = BeautifulSoup(open(f_pagina, "r", encoding="utf-8").read(), "html.parser")
    open("./estenograficas_limpias/" + f_pagina.replace("./estenograficas/",""), "w", encoding="utf-8").write(soup.get_text())

In [22]:
import os
from datetime import datetime
def normaliza(str_fecha):
    datetime_object = datetime.strptime(str_fecha[:6]+"20"+str_fecha[6:], '%d.%m.%Y')
    newformat = datetime_object.strftime('%Y-%m-%d')

    return newformat

def is_valid_date(str_fecha):
    try:
        datetime.strptime(str_fecha[:6]+"20"+str_fecha[6:], '%d.%m.%Y')
        return True
    except ValueError:
        return False

i = 0
fechas_conferencia = []
for f_pagina in glob.glob("./estenograficas_limpias/*"):
    texto = open(f_pagina, "r", encoding="utf-8").read()
    fechas_en_archivo = re.findall('[0-9]{2}\.[0-9]{2}\.[0-9]{2}', texto)
    #print(fechas_en_archivo)
    if len(fechas_en_archivo)==4 and is_valid_date(fechas_en_archivo[0]):
        if not os.path.isfile("./estenograficas_limpias_por_fecha/" + normaliza(fechas_en_archivo[0])):
            open("./estenograficas_limpias_por_fecha/" + normaliza(fechas_en_archivo[0]), "w", encoding='utf-8').write(texto)
            fechas_conferencia += [normaliza(fechas_en_archivo[0])]
        elif not os.path.isfile("./estenograficas_limpias_por_fecha/" + normaliza(fechas_en_archivo[0])+"_2da"):
            open("./estenograficas_limpias_por_fecha/" + normaliza(fechas_en_archivo[0]) + "_2da", "w", encoding='utf-8').write(texto)
            fechas_conferencia += [normaliza(fechas_en_archivo[0])]
            print("SEGUNDA: " + f_pagina)
        elif not os.path.isfile("./estenograficas_limpias_por_fecha/" + normaliza(fechas_en_archivo[0])+"_3ra"):
            open("./estenograficas_limpias_por_fecha/" + normaliza(fechas_en_archivo[0]) + "_3ra", "w", encoding='utf-8').write(texto)
            fechas_conferencia += [normaliza(fechas_en_archivo[0])]
            print("TERCERA: " + f_pagina)
    else:
        print("RARO: ", f_pagina)
        open("./raro/" + f_pagina[26:], "w", encoding='utf-8').write(texto)
        

RARO:  ./estenograficas_limpias/-08-1020-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-
SEGUNDA: ./estenograficas_limpias/-18-04-23-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-
SEGUNDA: ./estenograficas_limpias/-23-02-23-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-2-
SEGUNDA: ./estenograficas_limpias/-29-05-19-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-2-
SEGUNDA: ./estenograficas_limpias/-12-11-21-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-
RARO:  ./estenograficas_limpias/-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-67-
RARO:  ./estenograficas_limpias/-29-0920-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-man